# What is Semantic Web and RDF?

**RDF (Resource Description Framework)** is one of the three foundational [Semantic Web](https://en.wikipedia.org/wiki/Semantic_Web) technologies, the other two being SPARQL and OWL.

In particular, RDF is the data model of the Semantic Web. That means that all data in Semantic Web technologies is represented as RDF. If you store Semantic Web data, it's in RDF. If you query Semantic Web data (typically using SPARQL), it's RDF data. If you send Semantic Web data to your friend, it's RDF.

RDF data model is based upon the idea of making statements about resources (in particular web resources) in the form of *subject–predicate–object* expressions, known as [*triples*](https://en.wikipedia.org/wiki/Semantic_triple). The *subject* denotes the resource, and the *predicate* denotes traits or aspects of the resource, and expresses a relationship between the *subject* and the *object*.

For example, one way to represent the notion "The sky has the color blue" in RDF is as the triple: a **subject** denoting *"the sky"*, a **predicate** denoting *"has the color"*, and an **object** denoting *"blue"*. Therefore, RDF uses subject instead of object(or entity) in contrast to the typical approach of an entity–attribute–value model in object-oriented design: entity (sky), attribute (color), and value (blue).

![Image](http://dublincore.org/documents/2008/01/14/dc-rdf/rdfexamplefig.png)

Find out more: <br>
- http://fast.wistia.net/embed/iframe/8nm9xf4jip?popover=true <br>
- https://en.wikipedia.org/wiki/Resource_Description_Framework <br>
- https://www.cambridgesemantics.com/semantic-university/rdf-101 <br>
- http://www.cambridgesemantics.com/semantic-university/introduction-semantic-web-0

# RDF<->odML converter

Here we will explore RDF-odML and odML-RDF conversion in `odml/tools/rdf_converter.py` module.

Let's create the example odML document.

In [4]:
import odml
import datetime

doc = odml.Document(author="D. N. Adams",
                    date=datetime.date(1979, 10, 12))

# CREATE AND APPEND THE MAIN SECTIONs
doc.append(odml.Section(name="Arthur Philip Dent",
                           type="crew/person",
                           definition="Information on Arthur Dent"))

# SET NEW PARENT NODE
parent = doc['Arthur Philip Dent']


# APPEND PROPERTIES WITH VALUES
parent.append(odml.Property(name="Species",
                            value="Human",
                            dtype=odml.DType.string,
                            definition="Species to which subject belongs to"))

##RDFWriter class

RDFWriter class is used for conversion documents from odML to one of the supported RDF formats:<br>
'xml', 'pretty-xml', 'trix', 'n3', 'turtle', 'ttl', 'ntriples', 'nt', 'nt11', 'trig', 'json-ld'.<br>
Both one document or list of multiple documents can be passed to `RDFWriter()` constructor.

It's possible to get the output as a string.

In [5]:
from odml.tools.rdf_converter import RDFWriter

print(RDFWriter(doc).get_rdf_str('turtle'))

@prefix odml: <https://g-node.org/projects/odml-rdf#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

odml:Hub odml:hasDocument odml:db131aa9-c7ef-4df1-8b59-d538e7064917 .

<https://g-node.org/projects/odml-rdf#634a1560-60e4-4cc0-9123-7358583eac1e> a odml:Section ;
    odml:hasDefinition "Information on Arthur Dent" ;
    odml:hasName "Arthur Philip Dent" ;
    odml:hasProperty <https://g-node.org/projects/odml-rdf#6aee580d-6dbb-4a43-ad26-f8cba4722981> ;
    odml:hasType "crew/person" .

<https://g-node.org/projects/odml-rdf#6aee580d-6dbb-4a43-ad26-f8cba4722981> a odml:Property ;
    odml:hasDefinition "Species to which subject belongs to" ;
    odml:hasDtype "string" ;
    odml:hasName "Species" ;
    odml:hasValue odml:c7fa3b54-7b1b-4a43-9fa9-d6213f79cfe1 .

odml:c7fa3b54-7b1b-4a43-9fa9-d6213f79cfe1 a rdf:Bag 

Or write the output to the specified file.

In [17]:
import tempfile
import os

# Create temporary file
f = tempfile.NamedTemporaryFile(mode='w', suffix=".ttl")
path = f.name

# possible to use 'ttl' instead of 'turtle'
RDFWriter(doc).write_file(path, "ttl")

with open(path) as ff:
    data = ff.read()
    print(data)

f.close()

@prefix odml: <https://g-node.org/projects/odml-rdf#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

odml:Hub odml:hasDocument odml:db131aa9-c7ef-4df1-8b59-d538e7064917 .

<https://g-node.org/projects/odml-rdf#634a1560-60e4-4cc0-9123-7358583eac1e> a odml:Section ;
    odml:hasDefinition "Information on Arthur Dent" ;
    odml:hasName "Arthur Philip Dent" ;
    odml:hasProperty <https://g-node.org/projects/odml-rdf#6aee580d-6dbb-4a43-ad26-f8cba4722981> ;
    odml:hasType "crew/person" .

<https://g-node.org/projects/odml-rdf#6aee580d-6dbb-4a43-ad26-f8cba4722981> a odml:Property ;
    odml:hasDefinition "Species to which subject belongs to" ;
    odml:hasDtype "string" ;
    odml:hasName "Species" ;
    odml:hasValue odml:f52e8f94-8dcc-40db-b346-a539681fe463 .

odml:db131aa9-c7ef-4df1-8b59-d538e7064917 a odml:Doc

##RDFReader class

RDFReader class enables RDF to odML conversion.

There are 2 ways to obtain objects with converted odML documents:
- from **RDF file**  ( `RDFReader().from_file("/path_to_input_rdf", "rdf_format")` )
- from **RDF string**  ( `RDFReader().from_string("rdf file as a string", "rdf_format")` )

In [10]:
from odml.tools.rdf_converter import RDFReader

rdf_file = RDFWriter(doc).get_rdf_str('ttl')
odml_doc = RDFReader().from_string(rdf_file, "ttl")

print(odml_doc)

[<Doc None by D. N. Adams (1 sections)>]


In [19]:
# Create temporary file
rdf_file = tempfile.NamedTemporaryFile(mode='w', suffix=".ttl")
rdf_path = rdf_file.name
RDFWriter(doc).write_file(rdf_path, "ttl")

odml_doc = RDFReader().from_file(rdf_path, "ttl")

print(odml_doc)

[<Doc None by D. N. Adams (1 sections)>]


Another option is to write the output to one or multiple files. <br>
`RDFReader().write_file("/input_path", "rdf_format", "/output_path_to_file")`

In [21]:
# If RDF file contains one odML document, specify output path as file
odml_file = tempfile.NamedTemporaryFile(mode='w', suffix=".odml")
odml_path = odml_file.name

RDFReader().write_file(rdf_path, "ttl", odml_path)

with open(odml_path) as ff:
    data = ff.read()
    print(data)

<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet  type="text/xsl" href="odmlTerms.xsl"?>
<?xml-stylesheet  type="text/xsl" href="odml.xsl"?>
<odML version="1.1">
  <author>D. N. Adams</author>
  <id>db131aa9-c7ef-4df1-8b59-d538e7064917</id>
  <date>1979-10-12</date>
  <section>
    <name>Arthur Philip Dent</name>
    <id>634a1560-60e4-4cc0-9123-7358583eac1e</id>
    <type>crew/person</type>
    <definition>Information on Arthur Dent</definition>
    <property>
      <name>Species</name>
      <id>6aee580d-6dbb-4a43-ad26-f8cba4722981</id>
      <type>string</type>
      <definition>Species to which subject belongs to</definition>
      <value>[Human]</value>
    </property>
  </section>
</odML>



If RDF file contains several odML docs, specify output path as a directory.<br>
`RDFReader().write_file("/input_path", "rdf_format", "/output_path_to_directory")`

Module creates files in specified directory and writes parsed docs to them.
Example of created file: `/<dir_path>/doc_<id>.odml` (`<id>` - id of the document).